In [89]:
import pandas as pd

df = pd.read_csv('/Users/pablogandia/Desktop/ASIGNATURAS/IA SAMSUNG/Proyecto/Bias/Data/NewsBias.csv')
df.head()

,id,text,label
0,0,what got us through for many it was hobbies re...,leaning-left
1,1,new year s eve gatherings could accelerate ent...,leaning-left
2,2,entity traditions that you can still participa...,center
3,3,here is how countries around the world are rin...,leaning-right
4,4,police in entity said on thursday evening that...,left


In [90]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np


le = LabelEncoder()

texts = df['text'].astype(str).tolist()
labels = le.fit_transform(df['label'])

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_test  = np.array(X_test)

print(X_train.shape) 
print(X_test.shape)   
print(y_train.shape)  
print(y_test.shape)


(8603,)
(2151,)
(8603,)
(2151,)


In [93]:
import tensorflow as tf

max_tokens = 20000
max_len = 256

vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode='int',
    output_sequence_length=max_len
)
vectorize_layer.adapt(X_train)


def vectorize_text(text, label):
    return vectorize_layer(text), label

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = (
    train_ds
    .map(vectorize_text)          
    .shuffle(1000)
    .batch(32)                    
    .prefetch(tf.data.AUTOTUNE)
)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = (
    test_ds
    .map(vectorize_text)           
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)


In [94]:
vocab_size = max_tokens + 1
embedding_dim = 128
n_classes = len(np.unique(labels))

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    tf.keras.layers.SpatialDropout1D(0.2),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(n_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=5
)

Epoch 1/5
269/269 ━━━━━━━━━━━━━━━━━━━━ 78s 270ms/step - accuracy: 0.2635 - loss: 1.5786 - val_accuracy: 0.2873 - val_loss: 1.5681
Epoch 2/5
269/269 ━━━━━━━━━━━━━━━━━━━━ 77s 285ms/step - accuracy: 0.3019 - loss: 1.5305 - val_accuracy: 0.2789 - val_loss: 1.5913
Epoch 3/5
269/269 ━━━━━━━━━━━━━━━━━━━━ 75s 279ms/step - accuracy: 0.4224 - loss: 1.3451 - val_accuracy: 0.2603 - val_loss: 1.8351
Epoch 4/5
269/269 ━━━━━━━━━━━━━━━━━━━━ 73s 269ms/step - accuracy: 0.5816 - loss: 1.0537 - val_accuracy: 0.2757 - val_loss: 2.1697
Epoch 5/5
269/269 ━━━━━━━━━━━━━━━━━━━━ 82s 303ms/step - accuracy: 0.7139 - loss: 0.7675 - val_accuracy: 0.2710 - val_loss: 2.5970
